In [8]:
import numpy as np
from sklearn.neural_network import MLPClassifier
from tensorflow.keras.datasets import mnist
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import plot_confusion_matrix

import matplotlib.pyplot as plt

from tensorflow.keras.layers import Dense, Conv2D
from tensorflow.keras.layers import ReLU
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Softmax
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten

In [9]:
num_classes = 10
input_shape = (28, 28)


with np.load("../../Docs/PDF/DeepLearning/dataset/mnist.npz") as data:
  x_train = data['x_train']
  y_train = data['y_train']
  x_test = data['x_test']
  y_test = data['y_test']

In [10]:
# Normalize
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

In [11]:
class Sequentia:
    @staticmethod
    def build(inputs=(28, 28, 1), outputs=10):
        model = Sequential()

        f1, f2 = [16, 64]
        # First block
        model.add(Conv2D(f1, (3, 3), input_shape=inputs, padding="same"))
        model.add(ReLU())
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

        # Second block
        model.add(Conv2D(f2, (3, 3), padding="same"))
        model.add(ReLU())
        model.add(MaxPooling2D())

        # FC
        model.add(Flatten())
        model.add(Dense(300))
        model.add(ReLU())

        model.add(Dense(outputs))

        # lastly, define the soft-max classifier
        model.add(Softmax())

        return model

model = Sequentia.build()
model.summary()
        

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 28, 28, 16)        160       
                                                                 
 re_lu_3 (ReLU)              (None, 28, 28, 16)        0         
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 14, 14, 16)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 14, 14, 64)        9280      
                                                                 
 re_lu_4 (ReLU)              (None, 14, 14, 64)        0         
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 7, 7, 64)         0         
 2D)                                                  

In [16]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseTopKCategoricalAccuracy

optimizer = Adam(0.001)

model.compile(
    optimizer = optimizer,
    loss=SparseCategoricalCrossentropy(),
    metrics=[SparseTopKCategoricalAccuracy()],
)
epochs = 10
model.fit(x_train, y_train, epochs=epochs, verbose=1)

Epoch 1/10
1875/1875 [==============================] - 49s 26ms/step - loss: 0.1225 - sparse_top_k_categorical_accuracy: 0.9976
Epoch 2/10
1875/1875 [==============================] - 55s 29ms/step - loss: 0.0414 - sparse_top_k_categorical_accuracy: 0.9999
Epoch 3/10
1875/1875 [==============================] - 60s 32ms/step - loss: 0.0262 - sparse_top_k_categorical_accuracy: 1.0000
Epoch 4/10
1875/1875 [==============================] - 66s 35ms/step - loss: 0.0197 - sparse_top_k_categorical_accuracy: 1.0000
Epoch 5/10
1875/1875 [==============================] - 64s 34ms/step - loss: 0.0142 - sparse_top_k_categorical_accuracy: 1.0000
Epoch 6/10
1875/1875 [==============================] - 63s 33ms/step - loss: 0.0106 - sparse_top_k_categorical_accuracy: 1.0000
Epoch 7/10
1875/1875 [==============================] - 61s 33ms/step - loss: 0.0078 - sparse_top_k_categorical_accuracy: 1.0000
Epoch 8/10
1875/1875 [==============================] - 63s 34ms/step - loss: 0.0081 - sparse_top

In [17]:
model.save("./testeets.h5")
from tensorflow.keras.models import load_model
m = load_model("./testeets.h5") #só usar no lugar do model para rodar o teste
m.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 28, 28, 16)        160       
                                                                 
 re_lu_3 (ReLU)              (None, 28, 28, 16)        0         
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 14, 14, 16)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 14, 14, 64)        9280      
                                                                 
 re_lu_4 (ReLU)              (None, 14, 14, 64)        0         
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 7, 7, 64)         0         
 2D)                                                  

In [19]:
def score_accuracy(model, x_test, y_test):
    preds = model.predict(x_test)
    preds = np.argmax(preds, axis=1)
    accuracy = np.sum(y_test == preds)/len(y_test)*100
    print(f"Accuracy : {accuracy}%")

score_accuracy(model, x_test, y_test)

313/313 [==============================] - 2s 6ms/step
Accuracy : 99.15%
